# Part 1 - Creating a baseline

In this notebook we will create a simple yet important baseline so that we have an idea of how much our deep learning model improves the summaries. We use the ROUGE metric to measure the baseline.

In [1]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [51 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-310\tokenizers
  creating build\lib.win-amd64-cpython-310\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-310\tokenizers\models
  creating build\lib.win-amd64-cpython-310\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-310\tokenizers\decoders
  creating build\lib.win-amd64-cpython-310\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-310\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-310\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-31


  Using cached sagemaker-2.215.0-py3-none-any.whl.metadata (14 kB)
  Using cached transformers-4.6.1-py3-none-any.whl.metadata (45 kB)
  Using cached datasets-1.6.2-py3-none-any.whl.metadata (9.2 kB)
  Using cached huggingface_hub-0.0.8-py3-none-any.whl.metadata (8.7 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tqdm-4.49.0-py2.py3-none-any.whl.metadata (55 kB)
  Using cached boto3-1.16.43-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached botocore-1.19.52-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached s3fs-2024.3.1-py3-none-any.whl.metadata (1.6 kB)
  Using cache

In [ ]:
!pip install torch

In [1]:
import torch

In [ ]:
import scipy

In [2]:
torch.cuda.is_available()

False

In [2]:
!pip install rouge_score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24970 sha256=80c3ebbec4e5cbcd5c65c4b24fef3d816844c56fe26c25d9d1b10e1133ea5ff6
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import pandas as pd
df_test = pd.read_csv('data/test.csv')

In [4]:
df_test.head()

,text,summary
0,Consider the blow-up $X$ of $\mathbb{P}^3$ a...,Birational geometry of blow-ups of projective ...
1,"New deformed affine algebras A_{\hbar,\eta}(...","The algebra A_{\hbar,\eta}(\hat{g}) and Infini..."
2,"Currently, Deep Learning (DL) components wit...",Informed Machine Learning for Improved Similar...
3,Highly efficient and widely applicable worki...,Visible -Light-Gated Reconfigurable Rotation o...
4,"Using experiments and theory, we show that l...",Liquid Transport Due to Light Scattering


In [5]:
from datasets import load_metric
metric = load_metric("rouge")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11248\2721381364.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
C:\Users\ASUS\anaconda3\lib\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


We're copying this function from https://github.com/huggingface/transformers/blob/v4.6.1/examples/pytorch/summarization/run_summarization.py to ensure we always use the same metric calculation.

In [6]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

The summaries from the test dataset are the references

In [7]:
ref_summaries = list(df_test['summary'])

Now we cerate 3 baselines by comparing the reference summaries with the first sentence, the first 2 sentences, and the first 3 sentences in the abstract

In [8]:
import re
for i in range (3):
    candidate_summaries = list(df_test['text'].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i+1])))
    print(f"First {i+1} senctences: Scores {calc_rouge_scores(candidate_summaries, ref_summaries)}")

First 1 senctences: Scores {'rouge1': 31.3, 'rouge2': 15.5, 'rougeL': 26.4, 'rougeLsum': 26.4}
First 2 senctences: Scores {'rouge1': 23.9, 'rouge2': 11.4, 'rougeL': 19.2, 'rougeLsum': 19.2}
First 3 senctences: Scores {'rouge1': 19.8, 'rouge2': 9.6, 'rougeL': 15.7, 'rougeLsum': 15.7}


In [14]:
candidate_summaries[0]

'  Consider the blow-up $X$ of $\\mathbb{P}^3$ at 6 points in very general position and the 15 lines through the 6 points. We construct an infinite-order pseudo-automorphism $\\phi_X$ on $X$, induced by the complete linear system of a divisor of degree 13. The effective cone of $X$ has infinitely many extremal rays and hence, $X$ is not a Mori Dream Space.'

In [15]:
df_test['text'][0]

"  Consider the blow-up $X$ of $\\mathbb{P}^3$ at 6 points in very general position and the 15 lines through the 6 points. We construct an infinite-order pseudo-automorphism $\\phi_X$ on $X$, induced by the complete linear system of a divisor of degree 13. The effective cone of $X$ has infinitely many extremal rays and hence, $X$ is not a Mori Dream Space. The threefold $X$ has a unique anticanonical section which is a Jacobian K3 Kummer surface $S$ of Picard number 17. The restriction of $\\phi_X$ on $S$ realizes one of Keum's 192 infinite-order automorphisms of Jacobian K3 Kummer surfaces. In general, we show the blow-up of $\\mathbb{P}^n$ ($n\\geq 3$) at $(n+3)$ very general points and certain 9 lines through them is not Mori Dream, with infinitely many extremal effective divisors. As an application, for $n\\geq 7$, the blow-up of $\\overline{M}_{0,n}$ at a very general point has infinitely many extremal effective divisors. "